In [5]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-6aj9mn20
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-6aj9mn20
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=254bf9f2d0cf771a4c1a03fab12fd3c0056678ceec0875f5960c42c0340b45ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-wpmmevku/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [6]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [7]:
%%cu 
#include <stdio.h>
#include <stdlib.h>

//have to take this in row major format


#define N 2

void generateRandomArray(int* arrayA,int *arrayB) {
	srand((unsigned) time(NULL));
	for(int i = 0; i<N*N; ++i){
		arrayA[i] = ((int)rand()/(int)(RAND_MAX)) * 100;
		arrayB[i] = ((int)rand()/(int)(RAND_MAX)) * 100;
		
	}
}

__global__ void matrixAddColOnly(int *A,int *B,int *C){
    int colId = threadIdx.x+blockIdx.x*blockDim.x;

    if(colId <2){
        for(int i=0;i<2;i++){
            C[colId+i*2] = A[colId+i*2]+B[colId+i*2];
        }
    }

}

int main(){
    //in actuality a 2*2 matrix

    const size_t d_size = sizeof(int) * size_t(N*N);

    int *A = (int*) malloc(N*N * sizeof(int));
	  int *B = (int*) malloc(N*N * sizeof(int));
	  int *C = (int*) malloc(N*N * sizeof(int));
    //generateRandomArray(A,B);
    for(int i=0;i<N*N;i++){
        A[i]=4*i+1;
    }
    for(int i=0;i<N*N;i++){
        B[i]=5*i+2;
    }
    for(int i=0;i<N*N;i++){
        C[i]=0;
    }

    int *pA , *pB,*pC;
    cudaMalloc((void**)&pA,d_size);
    cudaMalloc((void**)&pB,d_size);
    cudaMalloc((void**)&pC,d_size);

    cudaMemcpy(pA,A,d_size,cudaMemcpyHostToDevice);
    cudaMemcpy(pB,B,d_size,cudaMemcpyHostToDevice);
    cudaMemcpy(pC,C,d_size,cudaMemcpyHostToDevice);
    int numBlocks=1; //cause using single block
    dim3 threadsPerBlock(N,N);
    dim3 blocksPerGrid(1,1);
    matrixAddColOnly<<<blocksPerGrid,threadsPerBlock>>>(pA,pB,pC);
    cudaMemcpy(C,pC,d_size,cudaMemcpyDeviceToHost);
    for(int i=0;i<N*N;i++){
        printf("A: =%d, ",A[i]);
    }
    printf("\n");
    for(int i=0;i<N*N;i++){
        printf("B: =%d, ",B[i]);
    }
    printf("\n");
    for(int i=0;i<N*N;i++){
        printf("C: =%d, ",C[i]);
    }
   

    cudaFree(pA);
    cudaFree(pB);
    cudaFree(pC);
    free(A);
    free(B);
    free(C);
    return 0;

}

A: =1, A: =5, A: =9, A: =13, 
B: =2, B: =7, B: =12, B: =17, 
C: =3, C: =12, C: =21, C: =30, 
